# Parquet For Data Processing

This is a demo of parquet data format and its capabilities for big data processing.
For this demo, we will be using pandas, sqlite, pyarrow and pyspark libraries to demonstrate the parquet capabilities.
The dataset that we will use is an sqlite dump of [wikibooks](https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset) from kaggle. It contains 270K chapters of wikibooks in 12 languages, but we will concentrate on the English version.

- pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools in Python.
- sqlite is an embedded SQL database engine, that uses the more traditional [B-Tree data-format](https://www.sqlite.org/fileformat2.html) for storage on disk 
- pyarrow is Python API of the [Apache Arrow](https://arrow.apache.org/) framework that defines an in-memory data representation and can read/write parquet, including conversion to pandas.
- pyspark is a Python API to the [Apache Spark Engine](https://spark.apache.org/), interfaces Python commands with a Java/Scala execution core, and thereby gives Python programmers access to the Parquet format as parquet is natively supported in spark.

In [1]:
import sqlite3